In [39]:
import pandas as pd
import numpy as np
import joblib
import streamlit as st
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier


In [10]:
df1 = pd.read_csv("C:/Users/pethk/Downloads/Early-stage-Ransomware-Detection-based-on-Pre-Attack-Internal-API-Calls-main/esrd_multiclass_dataset.csv")
    

In [9]:
df2 = pd.read_csv("C:/Users/pethk/Downloads/Early-stage-Ransomware-Detection-based-on-Pre-Attack-Internal-API-Calls-main/output_processed_2.csv")
    

In [11]:
print(df1.info())  # Check column types & missing values
print(df2.info())  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5203 entries, 0 to 5202
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   CreateProcessInternalW    5203 non-null   int64
 1   CreateServiceA            5203 non-null   int64
 2   CreateServiceW            5203 non-null   int64
 3   CryptExportKey            5203 non-null   int64
 4   CryptGenKey               5203 non-null   int64
 5   DeviceIoControl           5203 non-null   int64
 6   EnumServicesStatusA       5203 non-null   int64
 7   EnumServicesStatusW       5203 non-null   int64
 8   FindWindowA               5203 non-null   int64
 9   GetAdaptersAddresses      5203 non-null   int64
 10  GetComputerNameA          5203 non-null   int64
 11  GetComputerNameW          5203 non-null   int64
 12  GetDiskFreeSpaceExW       5203 non-null   int64
 13  GetDiskFreeSpaceW         5203 non-null   int64
 14  GlobalMemoryStatusEx      5203 non-null 

In [12]:
print(df1.head())  # Show first few rows
print(df2.head())


   CreateProcessInternalW  CreateServiceA  CreateServiceW  CryptExportKey  \
0                       0               0               0               0   
1                       0               0               0               0   
2                       1               0               0               0   
3                       0               0               0               0   
4                       1               0               0               0   

   CryptGenKey  DeviceIoControl  EnumServicesStatusA  EnumServicesStatusW  \
0            0                0                    0                    0   
1            0                0                    0                    0   
2            0                0                    0                    0   
3            0                0                    0                    0   
4            0                0                    0                    0   

   FindWindowA  GetAdaptersAddresses  ...  RegSetValueExA  RegSetValueExW 

In [13]:
print(df1.isnull().sum())  # Count missing values
print(df2.isnull().sum())




CreateProcessInternalW      0
CreateServiceA              0
CreateServiceW              0
CryptExportKey              0
CryptGenKey                 0
DeviceIoControl             0
EnumServicesStatusA         0
EnumServicesStatusW         0
FindWindowA                 0
GetAdaptersAddresses        0
GetComputerNameA            0
GetComputerNameW            0
GetDiskFreeSpaceExW         0
GetDiskFreeSpaceW           0
GlobalMemoryStatusEx        0
InternetOpenA               0
IsDebuggerPresent           0
LdrGetDllHandle             0
LookupPrivilegeValueW       0
MoveFileWithProgressW       0
NtAllocateVirtualMemory     0
NtCreateFile                0
NtCreateKey                 0
NtGetContextThread          0
NtMapViewOfSection          0
NtProtectVirtualMemory      0
NtQuerySystemInformation    0
NtResumeThread              0
NtSetContextThread          0
NtSetValueKey               0
NtTerminateProcess          0
NtUnmapViewOfSection        0
NtWriteFile                 0
Process32N

In [14]:
print(df1.duplicated().sum())  # Check duplicates
print(df2.duplicated().sum())

4285
4285


In [16]:
print(df2.columns)



Index(['CreateProcessInternalW', 'CreateServiceA', 'CreateServiceW',
       'CryptExportKey', 'CryptGenKey', 'DeviceIoControl',
       'EnumServicesStatusA', 'EnumServicesStatusW', 'FindWindowA',
       'GetAdaptersAddresses', 'GetComputerNameA', 'GetComputerNameW',
       'GetDiskFreeSpaceExW', 'GetDiskFreeSpaceW', 'GlobalMemoryStatusEx',
       'InternetOpenA', 'IsDebuggerPresent', 'LdrGetDllHandle',
       'LookupPrivilegeValueW', 'MoveFileWithProgressW',
       'NtAllocateVirtualMemory', 'NtCreateFile', 'NtCreateKey',
       'NtGetContextThread', 'NtMapViewOfSection', 'NtProtectVirtualMemory',
       'NtQuerySystemInformation', 'NtResumeThread', 'NtSetContextThread',
       'NtSetValueKey', 'NtTerminateProcess', 'NtUnmapViewOfSection',
       'NtWriteFile', 'Process32NextW', 'RegOpenKeyExA', 'RegOpenKeyExW',
       'RegSetValueExA', 'RegSetValueExW', 'SetFileAttributesW',
       'SetWindowsHookExA', 'SetWindowsHookExW', 'ShellExecuteExW',
       'WriteConsoleA', 'WriteConsoleW', 'W

In [17]:
print(df2["Family"].unique())  # Check unique values in 'Family'



[ 9 17 14  2  5  7  6  3 13  8 15 23  0]


In [18]:
print(df2["Family"].value_counts())


Family
17    450
2     450
14    450
5     450
23    450
15    450
9     443
0     432
6     377
3     359
13    331
8     295
7     266
Name: count, dtype: int64


In [24]:
X = df.drop(columns=['Family'])
y = df['Family']



In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


In [33]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [34]:
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

In [37]:

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [40]:
# Train XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
xgb_model.fit(X_train, y_train_encoded)  # Train using encoded labels
xgb_pred = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test_encoded, xgb_pred)  # Compare with encoded labels

C:\Users\pethk\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:18:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [41]:

st.success(f"XGBoost Accuracy: {xgb_accuracy:.2f}")

2025-03-01 15:18:21.357 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:18:21.579 
  command:

    streamlit run C:\Users\pethk\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-01 15:18:21.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [42]:
# Model Selection for Prediction
st.write("### Select Model for Ransomware Prediction")
model_choice = st.selectbox("Choose Model:", ["Random Forest", "XGBoost"])


2025-03-01 15:19:03.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:03.767 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [44]:
if st.button("Predict on Sample Data"):
    sample_data = X_test.iloc[0].values.reshape(1, -1)
    if model_choice == "Random Forest":
        prediction = rf_model.predict(sample_data)[0]
    else:
        prediction = xgb_model.predict(sample_data)[0]

    st.write(f"**Predicted Ransomware Family:** {prediction}")



2025-03-01 15:19:55.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:55.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:55.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:55.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 15:19:55.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
